In [1]:
import pandas as pd
import numpy as np

import stop_words
import nltk
from nltk.stem import  SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_ru = set(stopwords.words('russian'))
stop =  stop_ru | set(stop_words.get_stop_words('ru')) -set('год')

stemmer_ru = SnowballStemmer('russian')
stemmer_eng = SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from lightgbm import LGBMClassifier
from scipy.sparse import csr_matrix,hstack

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [ ]:
%%time
## Проверка точности модели - кросс валидация и просмотр точности по классам
## Для моделей с tf-idf
target_cat = 'class'

data = df
# new_data == вектора tf_idf

all_prediction = pd.DataFrame()
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit()
for tr, ts in ss.split(new_data, data[target_cat]):
    X_train, X_test, y_train,y_test = new_data.toarray()[tr], new_data.toarray()[ts], data[target_cat].iloc[tr], data[target_cat].iloc[ts]
    
    model.fit(X_train, y_train)
    prediction = model.predict_proba(X_test)
    prediction = pd.DataFrame(prediction)
    prediction['cat'] = y_test.values
    prediction['pred_cat'] = model.predict(X_test)
    prediction['index'] = ts


    all_prediction = pd.concat([prediction,all_prediction], axis = 0)
# all_prediction.to_csv('data/cross_val_result.csv', index = False)



from sklearn.metrics import f1_score, precision_score, recall_score
# all_prediction = pd.read_csv('cross_val_result.csv', index_col = 'index')

score = pd.DataFrame()
score['cat_id'] = data[target_cat].unique()
score['precision_score'] = score['cat_id'].apply(lambda x: precision_score(all_prediction['cat'], all_prediction['pred_cat'],labels = [x], average = 'micro'))
score['f1_score'] = score['cat_id'].apply(lambda x: f1_score(all_prediction['cat'], all_prediction['pred_cat'],labels = [x], average = 'micro'))
score['recall_score'] = score['cat_id'].apply(lambda x: recall_score(all_prediction['cat'], all_prediction['pred_cat'],labels = [x], average = 'micro'))

In [ ]:
score.to_csv('data/scores/'+model.__name__+'.csv', index = False)


In [ ]:
# Отображение скора

import seaborn as sns
def plot_confusion_matrix(y_test, prediction, class_names, figsize = (10,7), fontsize=14):

    
    matrix = pd.DataFrame(confusion_matrix(y_test, prediction))
    m=(matrix/matrix.sum()*10**2).astype(int)
    confusion_matrix = m.values
    
    
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




In [ ]:
plot_confusion_matrix(y_test, prediction, data[target_cat].unique())